In [20]:
import os
import torch
import random
from itertools import permutations
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets

In [2]:
filepath = "/kaggle/input/jigsaw/New folder"
permutationfilepath = "permutations500.pth"
modelsave = "/kaggle/working/jigsaw_encoder_3layer.pth"

if not os.path.exists(permutationfilepath):
    allpermutations = list(permutations(range(9)))
    random.shuffle(allpermutations)
    selectedpermutation = torch.tensor(allpermutations[:500])
    torch.save(selectedpermutation, permutationfilepath)
    print("Generated permutations500.pth")

Generated permutations500.pth


In [3]:
patchtransform = transforms.Compose([
    transforms.ToTensor(),
])


In [4]:
class Pretextdataset(Dataset):
    def __init__(self, rootdirectory, permutationfile, transformpatch):
        self.image_paths = [os.path.join(rootdirectory, f) for f in os.listdir(rootdirectory) if f.endswith(".jpg")]
        self.permutations = torch.load(permutationfile)
        self.transform_patch = transformpatch

    def croppatches(self, img):
        img = img.resize((255, 255))
        patches = []
        width, height = img.size
        newwidth, newheight = width // 3, height // 3
        for i in range(3):
            for j in range(3):
                patch = img.crop((j * newwidth, i * newheight, (j + 1) * newwidth, (i + 1) * newheight))
                patches.append(patch)
        return patches

    def __getitem__(self, index):
        img = Image.open(self.image_paths[index]).convert("RGB")
        patches = self.croppatches(img)
        permutation_index = random.randint(0, len(self.permutations) - 1)
        permutation = self.permutations[permutation_index]
        shuffled = [self.transform_patch(patches[i]) for i in permutation]
        return torch.stack(shuffled), permutation_index

    def __len__(self):
        return len(self.image_paths)

In [7]:
class Pretext(nn.Module):
    def __init__(self, outputdimension=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(128, outputdimension, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(4)   
        )

    def forward(self, x):
        return self.encoder(x)

In [11]:
class Fullyconnectedpretext(nn.Module):
    def __init__(self, numpermutations=500):
        super().__init__()
        self.encoder = Pretext()
        self.classifier = nn.Sequential(
            nn.Flatten(),                            
            nn.Linear(28800, 1024),         
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, numpermutations)
        )

    def forward(self, x):
        B, N, C, H, W = x.shape                    
        x = x.view(B * N, C, H, W)                 
        features = self.encoder(x)                   
        features = features.view(B, 28800)                 
        return self.classifier(features)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Available:", torch.cuda.is_available())
e=10
batchsize=128
learningrate=1e-3
N = 500
model = Fullyconnectedpretext(numpermutations=N).to(device)

GPU Available: True


In [8]:
dataset = Pretextdataset(filepath, permutationfilepath, patchtransform)
loader = DataLoader(dataset, batch_size=batchsize, shuffle=True, num_workers=0)
model = Fullyconnectedpretext(numpermutations=N).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learningrate)

In [9]:
from tqdm import tqdm
for epoch in range(e):
    model.train()
    total, correct, totalloss = 0, 0, 0.0

    for batch_index, (x, y) in enumerate(tqdm(loader, desc=f"Epoch {epoch+1}/{e}")):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        totalloss += loss.item()
        predictions = out.argmax(1)
        correct += (predictions == y).sum().item()
        total += y.size(0)

        if batch_index % 10 == 0:
            print(f"Batch {batch_index}/{len(loader)} | Loss: {loss.item():.4f}")

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{e} | Avg Loss: {totalloss/len(loader):.4f} | Accuracy: {accuracy:.2f}%\n")

torch.save(model.state_dict(), modelsave)
print(f"Model saved to {modelsave}")

Epoch 1/10:   0%|          | 1/425 [00:02<17:38,  2.50s/it]

Batch 0/425 | Loss: 6.2191


Epoch 1/10:   3%|▎         | 11/425 [00:19<11:47,  1.71s/it]

Batch 10/425 | Loss: 6.2100


Epoch 1/10:   5%|▍         | 21/425 [00:36<10:54,  1.62s/it]

...

Batch 410/425 | Loss: 0.1710


Epoch 10/10:  99%|█████████▉| 421/425 [06:32<00:03,  1.04it/s]

Batch 420/425 | Loss: 0.2117


Epoch 10/10: 100%|██████████| 425/425 [06:35<00:00,  1.07it/s]


Epoch 10/10 | Avg Loss: 0.2317 | Accuracy: 95.19%

Model saved to /kaggle/working/jigsaw_encoder_3layer.pth


In [12]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

class Downstream(nn.Module):
    def __init__(self, num_classes=38):
        super(Downstream, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),     
            nn.ReLU(),
            nn.MaxPool2d(2),                                

            nn.Conv2d(64, 128, kernel_size=3, padding=1),   
            nn.ReLU(),
            nn.MaxPool2d(2),                                

            nn.Conv2d(128, 128, kernel_size=3, padding=1),   
            nn.ReLU(),
            nn.MaxPool2d(4),                                

            nn.Conv2d(128, 256, kernel_size=3, padding=1),  
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),  
            nn.ReLU(),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),  
            nn.ReLU(),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),  
            nn.ReLU(),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),  
            nn.ReLU(),

            nn.MaxPool2d(4)                                 
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                                   
            nn.Linear(8192, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 38)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = Downstream(38)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [17]:
from collections import OrderedDict

statedictory = torch.load("/kaggle/input/pretextweights/pretextweights.pth")

newstatedirectory = OrderedDict()
for i, j in statedictory.items():
    if i.startswith("encoder.encoder."):  
        newkey = i.replace("encoder.encoder.", "encoder.")  
        newstatedirectory[newkey] = j

pretextmodel = Pretext()
pretextmodel.load_state_dict(newstatedirectory)

downstreammodel = Downstream(num_classes=38)

downstreammodel.features[0].weight.data = pretextmodel.encoder[0].weight.data.clone()
downstreammodel.features[0].bias.data   = pretextmodel.encoder[0].bias.data.clone()

downstreammodel.features[3].weight.data = pretextmodel.encoder[3].weight.data.clone()
downstreammodel.features[3].bias.data   = pretextmodel.encoder[3].bias.data.clone()

downstreammodel.features[6].weight.data = pretextmodel.encoder[6].weight.data.clone()
downstreammodel.features[6].bias.data   = pretextmodel.encoder[6].bias.data.clone()

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batchsize = 32
e = 10
learningrate = 1e-3

transformtensor = transforms.Compose([
    transforms.ToTensor()
])

traindirectory = "/kaggle/input/jigsaw-supervised/split_dataset/fold1/train"
valdirectory   = "/kaggle/input/jigsaw-supervised/split_dataset/fold1/val"


train_dataset = datasets.ImageFolder(traindirectory, transform=transformtensor)
val_dataset   = datasets.ImageFolder(valdirectory, transform=transformtensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

model = downstreammodel.to(device)

c = nn.CrossEntropyLoss()
o = torch.optim.Adam(model.parameters(), lr=learningrate)

for epoch in range(e):
    print(f"\nEpoch {epoch + 1}/{e}")
    model.train()
    trainpredictions,trainlabels = [],[]
    for x, y in tqdm(train_loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        o.zero_grad()
        out = model(x)
        loss = c(out, y)
        loss.backward()
        o.step()

        predictedclasses=out.argmax(1)          
        predictedclasses=predictedclasses.cpu()  
        predictedclasses=predictedclasses.numpy() 
        trainpredictions.extend(predictedclasses)   
        
        truelabels=y.cpu().numpy()
        trainlabels.extend(truelabels)   

    trainaccuracy= accuracy_score(trainlabels, trainpredictions)
    print(f"Train Accuracy:{trainaccuracy:.4f}")

    model.eval()
    valpredictions,vallabels = [],[]
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Validating"):
            x, y = x.to(device), y.to(device)
            out = model(x)
            guessclasses = out.argmax(1)
            guessclasses = guessclasses.cpu()
            guessclasses = guessclasses.numpy()
            valpredictions.extend(guessclasses)
            
            trueclasses = y.cpu().numpy()
            vallabels.extend(trueclasses)


    valaccuracy = accuracy_score(vallabels, valpredictions)
    print(f"Validation Accuracy: {valaccuracy:.4f}")
torch.save(model.state_dict(), "downstream_fold1.pth")


Epoch 1/10


Training: 100%|██████████| 905/905 [03:32<00:00,  4.25it/s]


Train Accuracy:0.3532


Validating: 100%|██████████| 453/453 [01:14<00:00,  6.07it/s]


Validation Accuracy: 0.5827

Epoch 2/10


...

Validation Accuracy: 0.8553

Epoch 10/10


Training: 100%|██████████| 905/905 [02:21<00:00,  6.38it/s]


Train Accuracy:0.8967


Validating: 100%|██████████| 453/453 [00:38<00:00, 11.72it/s]


Validation Accuracy: 0.8566


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batchsize = 32
e = 10
learningrate = 1e-3

transformtensor = transforms.Compose([
    transforms.ToTensor()
])

traindirectory = "/kaggle/input/jigsaw-supervised/split_dataset/fold2/train"
valdirectory   = "/kaggle/input/jigsaw-supervised/split_dataset/fold2/val"

train_dataset = datasets.ImageFolder(traindirectory, transform=transformtensor)
val_dataset   = datasets.ImageFolder(valdirectory, transform=transformtensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

model = downstreammodel.to(device)

model.load_state_dict(torch.load("/kaggle/working/downstream_fold1.pth", map_location=device))

c = nn.CrossEntropyLoss()
o = torch.optim.Adam(model.parameters(), lr=learningrate)

for epoch in range(e):
    print(f"\nEpoch {epoch + 1}/{e}")
    model.train()
    trainpredictions, trainlabels = [], []
    for x, y in tqdm(train_loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        o.zero_grad()
        out = model(x)
        loss = c(out, y)
        loss.backward()
        o.step()

        predictedclasses = out.argmax(1).cpu().numpy()
        trainpredictions.extend(predictedclasses)
        trainlabels.extend(y.cpu().numpy())

    trainaccuracy = accuracy_score(trainlabels, trainpredictions)
    print(f"Train Accuracy: {trainaccuracy:.4f}")

    model.eval()
    valpredictions, vallabels = [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Validating"):
            x, y = x.to(device), y.to(device)
            out = model(x)
            guessclasses = out.argmax(1)
            guessclasses = guessclasses.cpu()
            guessclasses = guessclasses.numpy()
            valpredictions.extend(guessclasses)
            
            trueclasses = y.cpu().numpy()
            vallabels.extend(trueclasses)

    valaccuracy = accuracy_score(vallabels, valpredictions)
    print(f"Validation Accuracy: {valaccuracy:.4f}")

torch.save(model.state_dict(), "downstream_fold2.pth")



Epoch 1/10


Training: 100%|██████████| 906/906 [04:34<00:00,  3.30it/s]


Train Accuracy: 0.8693


Validating: 100%|██████████| 453/453 [01:23<00:00,  5.42it/s]


Validation Accuracy: 0.9182

Epoch 2/10


...

Validation Accuracy: 0.9054

Epoch 10/10


Training: 100%|██████████| 906/906 [02:21<00:00,  6.38it/s]


Train Accuracy: 0.9474


Validating: 100%|██████████| 453/453 [00:41<00:00, 11.04it/s]

Validation Accuracy: 0.9308


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batchsize = 32
e = 10
learningrate = 1e-3

transformtensor = transforms.Compose([
    transforms.ToTensor()
])

traindirectory = "/kaggle/input/jigsaw-supervised/split_dataset/fold3/train"
valdirectory   = "/kaggle/input/jigsaw-supervised/split_dataset/fold3/val"

train_dataset = datasets.ImageFolder(traindirectory, transform=transformtensor)
val_dataset   = datasets.ImageFolder(valdirectory, transform=transformtensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

model = downstreammodel.to(device)

model.load_state_dict(torch.load("/kaggle/working/downstream_fold2.pth", map_location=device))

c = nn.CrossEntropyLoss()
o = torch.optim.Adam(model.parameters(), lr=learningrate)

for epoch in range(e):
    print(f"\nEpoch {epoch + 1}/{e}")
    model.train()
    trainpredictions, trainlabels = [], []
    for x, y in tqdm(train_loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        o.zero_grad()
        out = model(x)
        loss = c(out, y)
        loss.backward()
        o.step()

        predictedclasses = out.argmax(1).cpu().numpy()
        trainpredictions.extend(predictedclasses)
        trainlabels.extend(y.cpu().numpy())

    trainaccuracy = accuracy_score(trainlabels, trainpredictions)
    print(f"Train Accuracy: {trainaccuracy:.4f}")

    model.eval()
    valpredictions, vallabels = [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Validating"):
            x, y = x.to(device), y.to(device)
            out = model(x)
            guessclasses = out.argmax(1)
            guessclasses = guessclasses.cpu()
            guessclasses = guessclasses.numpy()
            valpredictions.extend(guessclasses)
            
            trueclasses = y.cpu().numpy()
            vallabels.extend(trueclasses)

    valaccuracy = accuracy_score(vallabels, valpredictions)
    print(f"Validation Accuracy: {valaccuracy:.4f}")

torch.save(model.state_dict(), "downstream_fold3.pth")



Epoch 1/10


Training: 100%|██████████| 906/906 [05:08<00:00,  2.94it/s]


Train Accuracy: 0.9160


Validating: 100%|██████████| 453/453 [02:01<00:00,  3.73it/s]


Validation Accuracy: 0.9643

Epoch 2/10


...

Validation Accuracy: 0.9398

Epoch 10/10


Training: 100%|██████████| 906/906 [02:19<00:00,  6.47it/s]


Train Accuracy: 0.9567


Validating: 100%|██████████| 453/453 [00:40<00:00, 11.24it/s]


Validation Accuracy: 0.9213


In [31]:
testdirectory = "/kaggle/input/jigsaw-supervised/split_dataset/test"

transformtensor = transforms.Compose([
    transforms.ToTensor()
])

testdataset = datasets.ImageFolder(testdirectory, transform=transformtensor)
testloader = DataLoader(testdataset, batch_size=32, shuffle=False, num_workers=0)

model = downstreammodel.to(device)
model.load_state_dict(torch.load("/kaggle/working/downstream_fold3.pth", map_location=device))
model.eval()

testpredictions, testlabels = [],[]

with torch.no_grad():
    for x, y in tqdm(testloader, desc="Testing"):
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        predictions = outputs.argmax(1)
        predictions=predictions.cpu()
        predictions=predictions.numpy()
        testpredictions.extend(predictions)
        testlabels.extend(y.cpu().numpy())

testaccuracy = accuracy_score(testlabels, testpredictions)
print(f"\nTest Accuracy: {testaccuracy:.4f}")

Testing: 100%|██████████| 340/340 [01:07<00:00,  5.04it/s]


Test Accuracy: 0.8592
